In [14]:
import pandas as pd
df = pd.read_csv('adidas_sales.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Invoice Id        9648 non-null   int64 
 1   Retailer          9648 non-null   object
 2   Retailer ID       9648 non-null   int64 
 3   Invoice Date      9648 non-null   object
 4   Region            9648 non-null   object
 5   State             9648 non-null   object
 6   City              9648 non-null   object
 7   Product           9648 non-null   object
 8   Price per Unit    9648 non-null   object
 9   Units Sold        9648 non-null   object
 10  Total Sales       9648 non-null   object
 11  Operating Profit  9648 non-null   object
 12  Operating Margin  9648 non-null   object
 13  Sales Method      9648 non-null   object
dtypes: int64(2), object(12)
memory usage: 1.0+ MB


In [16]:
# Membersihkan karakter seperti koma (,), dolar ($), dan persen (%) pada kolom yang relevan
df['Price per Unit'] = df['Price per Unit'].str.replace(',', '', regex=False).str.replace('$', '', regex=False).str.replace('%', '', regex=False)
df['Total Sales'] = df['Total Sales'].str.replace(',', '', regex=False).str.replace('$', '', regex=False).str.replace('%', '', regex=False)
df['Operating Profit'] = df['Operating Profit'].str.replace(',', '', regex=False).str.replace('$', '', regex=False).str.replace('%', '', regex=False)
df['Operating Margin'] = df['Operating Margin'].str.replace(',', '', regex=False).str.replace('$', '', regex=False).str.replace('%', '', regex=False)

# Mengonversi kolom ke tipe data yang sesuai:
# 'Units Sold' menjadi int, kolom lainnya menjadi float
df['Units Sold'] = pd.to_numeric(df['Units Sold'], errors='coerce', downcast='integer')  # Menjadikan integer
df['Price per Unit'] = pd.to_numeric(df['Price per Unit'], errors='coerce')  # Mengonversi ke float
df['Total Sales'] = pd.to_numeric(df['Total Sales'], errors='coerce')  # Mengonversi ke float
df['Operating Profit'] = pd.to_numeric(df['Operating Profit'], errors='coerce')  # Mengonversi ke float
df['Operating Margin'] = pd.to_numeric(df['Operating Margin'], errors='coerce')  # Mengonversi ke float

# Verifikasi tipe data kolom setelah konversi
print(df[['Price per Unit', 'Units Sold', 'Total Sales', 'Operating Profit', 'Operating Margin']].dtypes)


Price per Unit      float64
Units Sold          float64
Total Sales           int64
Operating Profit      int64
Operating Margin      int64
dtype: object


In [18]:
df.to_csv('P2M3_satrio_data_raw.csv')

In [19]:
from great_expectations.data_context import FileDataContext

# Membuat konteks data dengan direktori proyek saat ini ('./')
context = FileDataContext.create(project_root_dir='./')


In [20]:
# Tentukan nama Datasource yang unik
datasource_name = 'adidas-sales-dataset'  # Nama datasource yang sesuai
datasource = context.sources.add_pandas(datasource_name)

# Tentukan nama asset dan path ke data (lokal atau URL)
asset_name = 'adidas-sales-data'  # Nama asset untuk dataset penjualan Adidas
path_to_data = r'C:\Users\ThinkPad\OneDrive - Bina Nusantara\Documents\Hacktiv8\Milestone\p2-ftds021-hck-m3-satriotn\P2M3_satrio_data_raw.csv'  # Ganti dengan path lokal Anda

# Menambahkan asset berupa CSV
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Membangun batch request untuk memuat data
batch_request = asset.build_batch_request()


In [21]:
# Nama Expectation Suite yang sesuai
expectation_suite_name = 'expectation-adidas-sales'

# Menambahkan atau memperbarui Expectation Suite
context.add_or_update_expectation_suite(expectation_suite_name)

# Membuat validator menggunakan batch_request dan expectation_suite_name
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Memeriksa beberapa baris pertama dari data
validator.head()


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Unnamed: 0,Invoice Id,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,0,1,Foot Locker,1185732,1/1/2020,Northeast,New York,New York,Men's Street Footwear,50.0,NaN,600000,300000,50,In-store
1,1,2,Foot Locker,1185732,1/2/2020,Northeast,New York,New York,Men's Athletic Footwear,50.0,NaN,500000,150000,30,In-store
2,2,3,Foot Locker,1185732,1/3/2020,Northeast,New York,New York,Women's Street Footwear,40.0,NaN,400000,140000,35,In-store
3,3,4,Foot Locker,1185732,1/4/2020,Northeast,New York,New York,Women's Athletic Footwear,45.0,850.0,382500,133875,35,In-store
4,4,5,Foot Locker,1185732,1/5/2020,Northeast,New York,New York,Men's Apparel,60.0,900.0,540000,162000,30,In-store


In [22]:
# 1. Expectation for Uniqueness
validator.expect_column_values_to_be_unique('Invoice Id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [23]:
validator.expect_column_values_to_be_between(
    column='Units Sold', min_value=0, max_value=10000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 53,
    "missing_percent": 0.5493366500829188,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [27]:
# 3. Expectation for Set Membership
valid_regions = ['Northeast','South','West','Midwest','Southeast']
validator.expect_column_values_to_be_in_set('Region', valid_regions)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [30]:
# 4. Expectation for Type List
validator.expect_column_values_to_be_in_type_list('Units Sold', ['float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [31]:
# 5. Expectation for Non-Null Values
validator.expect_column_values_to_not_be_null('Product')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [38]:
# 6. Expectation for Value Length
validator.expect_column_values_to_be_in_set('Sales Method', ['In-store', 'Outlet', 'Online'])


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [39]:
# 7. Expectation for Column Mean
df['Operating Profit'] = pd.to_numeric(df['Operating Profit'], errors='coerce')
validator.expect_column_mean_to_be_between('Operating Profit', min_value=1000)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 34425.26907131012
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [40]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

In [41]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [42]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Build data docs
context.build_data_docs()

{'local_site': 'file://c:\\Users\\ThinkPad\\OneDrive - Bina Nusantara\\Documents\\Hacktiv8\\Milestone\\p2-ftds021-hck-m3-satriotn\\gx\\uncommitted/data_docs/local_site/index.html'}